In [1]:
from newsapi import NewsApiClient
import pandas as pd
from pandas.io.json import json_normalize

In [3]:
from retrying import retry

In [4]:
SDN = pd.read_csv('SDN.csv', header = None, names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L'])
SP = pd.read_csv('trial-sp 100.csv')

In [5]:
SDN.head(5)

,A,B,C,D,E,F,G,H,I,J,K,L
0,36,AEROCARIBBEAN AIRLINES,-0-,CUBA,-0-,-0-,-0-,-0-,-0-,-0-,-0-,-0-
1,173,"ANGLO-CARIBBEAN CO., LTD.",-0-,CUBA,-0-,-0-,-0-,-0-,-0-,-0-,-0-,-0-
2,306,BANCO NACIONAL DE CUBA,-0-,CUBA,-0-,-0-,-0-,-0-,-0-,-0-,-0-,a.k.a. 'BNC'.
3,424,BOUTIQUE LA MAISON,-0-,CUBA,-0-,-0-,-0-,-0-,-0-,-0-,-0-,-0-
4,475,CASA DE CUBA,-0-,CUBA,-0-,-0-,-0-,-0-,-0-,-0-,-0-,-0-


In [6]:
SP.head(5)

,Symbol,Description,Category2,Category3,GICS Sector,Market cap,Dividend yield,Price to TTM earnings,Price to TTM sales,Price to book value,Action
0,AAPL,Apple Inc,Common stocks,Large cap,Information Technology,"$2,604,419,589,190",0.55%,26.07,6.88,36.44,Analyze
1,MSFT,Microsoft Corp,Common stocks,Large cap,Information Technology,"$2,207,002,507,739",0.80%,31.09,11.94,13.83,Analyze
2,GOOG,Alphabet Inc Class C,Common stocks,Large cap,Communication Services,"$1,805,278,992,900",0.00%,11.18,7.01,3.38,Analyze
3,GOOGL,Alphabet Inc Class A,Common stocks,Large cap,Communication Services,"$1,805,278,992,900",0.00%,10.59,7.01,3.20,Analyze
4,AMZN,Amazon.Com Inc.,Common stocks,Large cap,Consumer Discretionary,"$1,558,122,290,973",0.00%,47.96,3.32,11.58,Analyze


In [5]:
newsapi = NewsApiClient(api_key='newsapikey(the key cannot be published here)')

In [8]:
SDNco = SDN['B']
SPco = SP['Description']

In [9]:
SPco30 = SPco[0:30,]

In [18]:
SDNco

0                 AEROCARIBBEAN AIRLINES
1              ANGLO-CARIBBEAN CO., LTD.
2                 BANCO NACIONAL DE CUBA
3                     BOUTIQUE LA MAISON
4                           CASA DE CUBA
                      ...               
9357        KOVALCHUK, Kira Valentinovna
9358     KNYAGININ, Vladimir Nikolaevich
9359    KOVALCHUK, Tatyana Aleksandrovna
9360          KOVALCHUK, Boris Yurievich
9361                                 NaN
Name: B, Length: 9362, dtype: object

In [13]:
SPco30

0                           Apple Inc
1                      Microsoft Corp
2                Alphabet Inc Class C
3                Alphabet Inc Class A
4                     Amazon.Com Inc.
5                           Tesla Inc
6     Berkshire Hathaway Inc. Class B
7                         Nvidia Corp
8                Facebook Inc Class A
9              Unitedhealth Group Inc
10                   Visa Inc Class A
11                  Johnson & Johnson
12               JP Morgan Chase & Co
13                 Walmart Stores Inc
14                   Procter & Gamble
15                     Home Depot Inc
16               Bank of America Corp
17             Mastercard Inc Class A
18                   Exxon Mobil Corp
19                       Chevron Corp
20                         Pfizer Inc
21                         Abbvie Inc
22                          Eli Lilly
23                          Coca-Cola
24                        Walt Disney
25                      Broadcom Inc.
26          

In [ ]:
SDN_russia = SDN[SDN.D.str.contains('RUSSIA', na=False)].iloc[:,1]

In [ ]:
SDN_russia

3591        ROTENBERG, Arkady Romanovich
3593         ROTENBERG, Boris Romanovich
3601       KOVALCHUK, Yuri Valentinovich
3604                    CHEMEZOV, Sergei
3605           IVANOV, Sergei Borisovich
                      ...               
9356       KOVALCHUK, Stepan Kirillovich
9357        KOVALCHUK, Kira Valentinovna
9358     KNYAGININ, Vladimir Nikolaevich
9359    KOVALCHUK, Tatyana Aleksandrovna
9360          KOVALCHUK, Boris Yurievich
Name: B, Length: 306, dtype: object

In [16]:
class spco_news: 
    def __init__(self, name, news): 
        self.name = name 
        self.news = news

#get all news with 2 keywords
#here we only choose 3 companies in SP500 for NER trial, can modify'[3:5]''[:30]' to change the companies we choose
news_list_spco = list()
for k in SPco30[3:5]:
    news_list = list()
    for i in SDN_russia[:30]:
        key = str(k+ ' + ' + i)
        news = newsapi.get_everything(q= key,
                              from_param='2018-03-01',
                              to='2022-03-01',
                              language='en',
                              sort_by='relevancy',
                                page_size=50,
                                     page=1)
        if news['totalResults'] != 0:
            news_articles = pd.DataFrame(news['articles'])
            news = news_articles.loc[:,['content']]
            news['SDN'] = i
            news_list.append(news)
    if bool(news_list):
        news_list_spco.append(spco_news(k,news_list))

news_list_spco

[<__main__.spco_news at 0x28b9e2f0eb0>, <__main__.spco_news at 0x28b9e5d0580>]

In [17]:
df_list = list()
for i in news_list_spco:
    temp_df = pd.concat(i.news,ignore_index = True)
    temp_df['spco'] = i.name
    df_list.append(temp_df)

df_output = pd.concat(df_list,ignore_index = True)
df_output

,content,SDN,spco
0,(Bloomberg) -- Food-delivery company Deliveroo...,SMP BANK,Alphabet Inc Class A
1,<table><tr><td>-rw-r--r--</td><td>Documentatio...,SMP BANK,Alphabet Inc Class A
2,Yandex (NASDAQ: YNDX) is Russia's premier high...,VTB BANK PUBLIC JOINT STOCK COMPANY,Alphabet Inc Class A
3,<table><tr><td>-rw-r--r--</td><td>Documentatio...,VTB BANK PUBLIC JOINT STOCK COMPANY,Alphabet Inc Class A
4,<table><tr><td>-rw-r--r--</td><td>Documentatio...,LIMITED LIABILITY COMPANY VTB DC,Alphabet Inc Class A
5,(Bloomberg) -- Food-delivery company Deliveroo...,SMP BANK,Amazon.Com Inc.
6,This report describes and evaluates the global...,SMP BANK,Amazon.Com Inc.
7,Skip to content\r\nECS (Elitegroup Computer Sy...,SMP BANK,Amazon.Com Inc.
8,KVM: s390: Fix for 5.7 and maintainer update\r...,SMP BANK,Amazon.Com Inc.
9,Pull char/misc driver updates from Greg KH:\r\...,SMP BANK,Amazon.Com Inc.


In [19]:
df_output.to_csv('SP3-Russia.csv')